In [7]:
# 측정소별 실시간 측정정보 조회 (매시 15분 내외 업데이트)
base_url = "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty"
with open('keys/에어코리아api2.txt') as file:
    service_key = file.read() 

In [1]:
import requests
import pandas as pd
from draw_map import  BORDER_LINES, drawKorea, drawKoreaMinus
import matplotlib.pyplot as plt

# 측정소별 실시간 측정정보 조회 (매시 15분 내외 업데이트)
base_url = "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty"
with open('keys/에어코리아api4.txt') as file:
    service_key = file.read()       # 발급받은 에어코리아 API 키 입력
    
# 예제 데이터프레임 생성
station_Name = pd.read_csv('data/전국 대기측정소_완료.csv', encoding='utf-8')
map_raw = pd.read_excel('data/draw_korea_raw(2021).xlsx')
pop = pd.read_csv('data/위도경도_최종.csv', encoding='utf-8')

# 완성코드
def get_air_quality():
    # 결과를 저장할 리스트
    results_list = []
    
    # 각 측정소에 대한 데이터 수집
    for index, row in station_Name.iterrows():
        # 웹 요청시 같이 전달될 데이터 = 요청 메시지
        params = {
            'serviceKey' : service_key,
            'returnType' : 'JSON',
            'numOfRows' : 30,
            'pageNo' : 1,
            'stationName' : row['측정소명'],  # 각 행의 측정소명을 선택
            'dataTerm' : 'DAILY',
            'ver' : "1.4"  # 오퍼레이션 버전
        }

        res = requests.get(url=base_url , params=params)

        # 응답 데이터 정리
        data = res.json()['response']['body']['items']
        
        # 데이터가 존재하면 처리
        if data:
            # 데이터를 날짜와 시간에 대한 기준으로 정렬
            sorted_data = sorted(data, key=lambda x: x['dataTime'], reverse=True)

            # 최신 데이터 선택
            latest_data = sorted_data[0]

            # 최종 데이터 생성
            result = {
                '날짜': latest_data['dataTime'],
                '이름': latest_data['stationName'],
                '측정망 정보': latest_data['mangName'],
                '아황산가스 농도': latest_data['so2Value'] + "ppm",
                '일산화탄소 농도': latest_data['coValue'] + "ppm",
                '오존 농도': latest_data['o3Value'] + "ppm",
                '이산화질소 농도': latest_data['no2Value'] + "ppm",
                '미세먼지(PM10) 농도': latest_data['pm10Value'] + "ug/m³",
                '초미세먼지(PM2.5) 농도': latest_data['pm25Value'] + "ug/m³"
            }

            # 결과 리스트에 추가
            results_list.append(result)

    # result_list에서 미세먼지농도값만 pm10_values에 저장
    pm10_values = [entry['미세먼지(PM10) 농도'] for entry in results_list]
    
    
    pop = [int(value.split('ug/m³')[0]) for value in pm10_values]
    # 단위를 빼고 저장
    data = []
    for value in pm10_values:
        # 값이 '-'인 경우 0으로 처리하거나 다른 처리를 수행할 수 있습니다.
        if value != '-':
            data.append(int(value.split('ug/m³')[0]))
        else:
            data.append(0)  # 또는 다른 값을 넣어도 됩니다.
    
    map_raw.stack()[:10]
    map = pd.DataFrame(map_raw.stack())
    map.reset_index(inplace=True)
    map.columns = ['y','x','ID']
    pop = pd.merge(pop, map)
    
    # 샘플 데이터 생성
    air = {'ID': pop['ID'].tolist(),
            'Value': data}
    df = pd.DataFrame(air)

    # 지도 데이터와 샘플 데이터를 병합합니다.
    merge = pop.merge(df, how='left', left_on='ID', right_on='ID')
    
    # 카토그램 그리기
    drawKorea('미세먼지(PM10) 농도', merge, 'Blues')

# 공기 품질 데이터를 얻기 위해 함수 호출
get_air_quality()

ValueError: invalid literal for int() with base 10: '-'

In [2]:
import requests
import pandas as pd
from draw_map import  BORDER_LINES, drawKorea, drawKoreaMinus
import matplotlib.pyplot as plt

# 완성코드
def draw_choropleth_map(map_raw, df):
    map_raw.stack()[:10]
    map_df = pd.DataFrame(map_raw.stack())
    map_df.reset_index(inplace=True)
    map_df.columns = ['y', 'x', 'ID']
    df = pd.merge(df, map_df)
    plt.show()

def get_air_quality():
    # 결과를 저장할 리스트
    results_list = []
    
    # 각 측정소에 대한 데이터 수집
    for index, row in station_Name.iterrows():
        # 웹 요청시 같이 전달될 데이터 = 요청 메시지
        params = {
            'serviceKey' : service_key,
            'returnType' : 'JSON',
            'numOfRows' : 30,
            'pageNo' : 1,
            'stationName' : row['측정소명'],  # 각 행의 측정소명을 선택
            'dataTerm' : 'DAILY',
            'ver' : "1.4"  # 오퍼레이션 버전
        }

        res = requests.get(url=base_url , params=params)

        # 응답 데이터 정리
        data = res.json()['response']['body']['items']
        
        # 데이터가 존재하면 처리
        if data:
            # 데이터를 날짜와 시간에 대한 기준으로 정렬
            sorted_data = sorted(data, key=lambda x: x['dataTime'], reverse=True)

            # 최신 데이터 선택
            latest_data = sorted_data[0]

            # 최종 데이터 생성
            result = {
                '날짜': latest_data['dataTime'],
                '이름': latest_data['stationName'],
                '측정망 정보': latest_data['mangName'],
                '아황산가스 농도': latest_data['so2Value'] + "ppm",
                '일산화탄소 농도': latest_data['coValue'] + "ppm",
                '오존 농도': latest_data['o3Value'] + "ppm",
                '이산화질소 농도': latest_data['no2Value'] + "ppm",
                '미세먼지(PM10) 농도': latest_data['pm10Value'] + "ug/m³",
                '초미세먼지(PM2.5) 농도': latest_data['pm25Value'] + "ug/m³"
            }

            # 결과 리스트에 추가
            results_list.append(result)

    # result_list에서 미세먼지농도값만 pm10_values에 저장
    pm10_values = [entry['미세먼지(PM10) 농도'] for entry in results_list]
    # 단위빼서 저장
    pop = [int(value.split('ug/m³')[0]) for value in pm10_values]
    
    # 샘플 데이터 생성
    air = {'ID': ['강동구','강남구','강서구','영등포구'],
            'Value': data}
    df = pd.DataFrame(pop)

    # 지도 데이터와 샘플 데이터를 병합합니다.
    world_merged = map_raw.merge(df, how='left', left_on='ID', right_on='ID')


    # 비어있는 값은 0으로 채웁니다.
    world_merged['Value'] = world_merged['Value'].fillna(0)

    # 카토그램을 그립니다.
    draw_choropleth_map(world_merged, df)
    
# 함수 호출
get_air_quality()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
import requests
import pandas as pd
from draw_map import BORDER_LINES, drawKorea, drawKoreaMinus
import matplotlib.pyplot as plt

# 완성코드
def draw_choropleth_map(map_raw, df):
    map_raw.stack()[:10]
    map_df = pd.DataFrame(map_raw.stack())
    map_df.reset_index(inplace=True)
    map_df.columns = ['y', 'x', 'ID']
    df = pd.merge(df, map_df)
    plt.show()

def get_air_quality(pop, map_raw):
    # 결과를 저장할 리스트
    results_list = []
    
    # 각 측정소에 대한 데이터 수집
    for index, row in station_Name.iterrows():
        # 웹 요청시 같이 전달될 데이터 = 요청 메시지
        params = {
            'serviceKey' : service_key,
            'returnType' : 'JSON',
            'numOfRows' : 30,
            'pageNo' : 1,
            'stationName' : row['측정소명'],  # 각 행의 측정소명을 선택
            'dataTerm' : 'DAILY',
            'ver' : "1.4"  # 오퍼레이션 버전
        }

        res = requests.get(url=base_url , params=params)

        # 응답 데이터 정리
        data = res.json()['response']['body']['items']
        
        # 데이터가 존재하면 처리
        if data:
            # 데이터를 날짜와 시간에 대한 기준으로 정렬
            sorted_data = sorted(data, key=lambda x: x['dataTime'], reverse=True)

            # 최신 데이터 선택
            latest_data = sorted_data[0]

            # 최종 데이터 생성
            result = {
                '날짜': latest_data['dataTime'],
                '이름': latest_data['stationName'],
                '측정망 정보': latest_data['mangName'],
                '아황산가스 농도': latest_data['so2Value'] + "ppm",
                '일산화탄소 농도': latest_data['coValue'] + "ppm",
                '오존 농도': latest_data['o3Value'] + "ppm",
                '이산화질소 농도': latest_data['no2Value'] + "ppm",
                '미세먼지(PM10) 농도': latest_data['pm10Value'] + "ug/m³",
                '초미세먼지(PM2.5) 농도': latest_data['pm25Value'] + "ug/m³"
            }

            # 결과 리스트에 추가
            results_list.append(result)

    # result_list에서 미세먼지농도값만 pm10_values에 저장
    pm10_values = [entry['미세먼지(PM10) 농도'] for entry in results_list]
    
    # 단위를 빼고 저장
    pop_data = [int(value.split('ug/m³')[0]) if value.replace('-', '').replace('.', '').isdigit() else 0 for value in pm10_values]

    
    # 샘플 데이터 생성
    pop_df = pd.DataFrame({'ID': ['강동구', '강남구', '강서구', '영등포구'],
                            'Value': pop_data})

    # 지도 데이터와 샘플 데이터를 병합합니다.
    world_merged = map_raw.merge(pop_df, how='left', left_on='ID', right_on='ID')

    # 비어있는 값은 0으로 채웁니다.
    world_merged['Value'] = world_merged['Value'].fillna(0)

    # 카토그램을 그립니다.
    draw_choropleth_map(world_merged, pop_df)
    
# 함수 호출
get_air_quality(pop, map_raw)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)